# Генерация комментариев к посту в социальной сети

## Подготовка 

In [1]:
!pip install transformers
!pip install pickle5
!pip install tensorboardX

     |████████████████████████████████| 2.3MB 3.0MB/s 
     |████████████████████████████████| 3.3MB 22.7MB/s 
     |████████████████████████████████| 901kB 30.5MB/s 
     |████████████████████████████████| 133kB 2.9MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219245 sha256=8fb66f46b70a20fa4c2488a1c2e3644b683460fc0dbc26fbe393058d95e593a6
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5
     |████████████████████████████████| 122kB 2.9MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
from transformers import get_linear_schedule_with_warmup
from transformers import GPT2Config,AdamW
from transformers import AutoTokenizer, AutoModel
import pickle5 as pickle
from tqdm.notebook import tqdm
import json
from tensorboardX import SummaryWriter
import torch.nn.functional as F
import os
from datetime import datetime
import random
from random import randint
from transformers import AutoModelForCausalLM
import gc
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
from tqdm.notebook import tnrange, tqdm
import numpy as np
from random import shuffle
import re

In [4]:
# tokenizer = AutoTokenizer.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2") 
# tokenizer = AutoTokenizer.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2") 
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
num_add_toks = tokenizer.add_special_tokens(special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
POST_CNT = 5880
TEST_CNT = 20
VAL_CNT = 10
DATA_DIR = f'drive/MyDrive/Colab Notebooks/neuro_data/vk_{POST_CNT}.pickle'
IN_TOKENS = 768

## Данные

In [7]:
with open(DATA_DIR, 'rb') as handle:
    vk_comments = pickle.load(handle)

len(vk_comments)

5880

In [8]:
for_test_ids = []
ids_to_items = {}
i = 0
train_ready = {}

for post, cmts in tqdm(vk_comments):
  post_tok = tokenizer.encode(post)
  for_test_ids.append(i)
  for cmt in cmts:
    text = tokenizer.encode(tokenizer.pad_token) * IN_TOKENS

    cmt_tok = tokenizer.encode(cmt)
    content = post_tok + tokenizer.encode(tokenizer.sep_token) + cmt_tok
    if len(content) > IN_TOKENS or len(cmt_tok) == 0 or len(post_tok) == 0:
        print(f'{i};', end='')
        continue
    text[:len(content)] = content
    ids_to_items[i] = {'post': post_tok, 'comment': cmt_tok}
    train_ready[i] = {'post_com': text, 'post_len': len(post_tok)}
    i += 1

16556;127826;128421;129141;131094;132660;134986;139179;193885;


In [9]:
# средняя длина комментария
sum([len(ids_to_items[key]['comment']) for key in ids_to_items]) / len(ids_to_items)
# медиана
np.median(sorted([len(ids_to_items[key]['comment']) for key in ids_to_items]))

14.0

In [10]:
with open('post_comment_encoded.json', 'w') as f:
    json.dump(ids_to_items, f)

all_ids = set([i for i in train_ready.keys()]) 
max_idx = len(all_ids) - 1

In [11]:
test_ids = []
for i in range(POST_CNT - TEST_CNT, len(for_test_ids) - 1):
  j = for_test_ids[i]
  group = set()
  while j < for_test_ids[i+1]:
    group.add(j)
    j += 1
  all_ids = all_ids.difference(group)
  test_ids.append(group)
j = for_test_ids[-1]
group = set()

while j <= max_idx:
  group.add(j)
  j += 1
all_ids = all_ids.difference(group)
test_ids.append(group) # test_ids = [(idx1, idx2, ..)] - tupl'ы индексов для одного поста

len(test_ids)

20

тоже для валидации

In [12]:
valid_ids = []
for i in range(POST_CNT - TEST_CNT - VAL_CNT, POST_CNT - TEST_CNT):
  j = for_test_ids[i]
  group = set()
  while j < for_test_ids[i+1]:
    group.add(j)
    j += 1
  all_ids = all_ids.difference(group)
  valid_ids.append(group)

In [13]:
train_ids = []
for i in range(0, POST_CNT - TEST_CNT - VAL_CNT):
  j = for_test_ids[i]
  group = set()
  while j < for_test_ids[i+1]:
    group.add(j)
    j += 1
  all_ids = all_ids.difference(group)
  train_ids.append(group)

In [14]:
assert len(all_ids) == 0
len(for_test_ids) 

5880

**<>_dataset = {idx: [ {'post_com': ..., 'post_len': ...} , {} , ..] }** - индекс_поста -> [{пост+коммент, длина}, ..]

In [15]:
train_dataset = {}
i = 0

for gr in train_ids:
  group = []
  for ind in gr:
    group.append(train_ready[ind])
  train_dataset[i] = group
  i += 1
valid_dataset = {}

i = 0
for gr in valid_ids:
  group = []
  for ind in gr:
    group.append(train_ready[ind])
  valid_dataset[i] = group
  i += 1
test_dataset = {}

i = 0
for gr in test_ids:
  group = []
  for ind in gr:
    group.append(train_ready[ind])
  test_dataset[i] = group
  i += 1

вообще **<>_dataset_paired = idx -> {'post_com':.., 'post_len': ...}** для всех (очень*30 долго), но попробовала для каждого поста брать 1 рандомный комментарий (все равно долго)

In [16]:
train_dataset_paired = {}
i = 0
for gr in train_dataset:
  for item in train_dataset[gr]:
    train_dataset_paired[i] = item
    i += 1
    
valid_dataset_paired = {}
i = 0
for gr in valid_dataset:
  for item in valid_dataset[gr]:
    valid_dataset_paired[i] = item
    i += 1

## Модель

In [17]:
device = 'cuda:0'

In [18]:
ignore_idx = tokenizer.pad_token_id
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")
#model = AutoModelForCausalLM.from_pretrained("Grossmend/rudialogpt3_medium_based_on_gpt2")
#model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
#sberbank-ai/rugpt3medium_based_on_gpt2
#model = torch.load('drive/MyDrive/Colab Notebooks/rus01.pth')

In [19]:
print(len(tokenizer))
model.resize_token_embeddings(len(tokenizer))

50260


Embedding(50260, 768)

In [21]:
model.to(torch.device(device))

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [22]:
train_sampler = RandomSampler(train_dataset_paired)
train_dl = DataLoader(train_dataset_paired, sampler=train_sampler, batch_size=1, num_workers=2)

In [23]:
loss_fct = CrossEntropyLoss(ignore_index=ignore_idx) 
optimizer = AdamW(model.parameters(),lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer,100,80000)

In [24]:
CP_PATH = 'drive/MyDrive/Colab Notebooks/neuro_data/cp/checkpoint.ph'
END_PATH = 'drive/MyDrive/Colab Notebooks/neuro_data/cp/res.bin'
SEQ_LEN = 100
TEMP = 1.5

In [25]:
state = torch.load('drive/MyDrive/Colab Notebooks/neuro_data/cp/checkpoint_15.ph') # CP_PATH 

In [26]:
optimizer.load_state_dict(state['optimizer_state_dict'])
model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

## Fine-tuning

In [27]:
seps = set('.!?')
garb = set(' .!?,')

def cut(s, num=2):
    if num <= 0:
        num = 2

    start = 0
    for symb in s:
        if symb not in garb:
            break
        start += 1

    end = start
    while end < len(s):
        if num == 0:
            break

        while s[end] in seps and end + 1 < len(s) and s[end + 1] in seps:
            end += 1

        if s[end] in seps:
            num -= 1
        end += 1
    return s[start:end]

In [28]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_seq(model, context, length, device, temperature=TEMP, top_k=0, top_p=0.0):
    """ Generates a sequence of tokens 
        Args:
            model: gpt/gpt2 model
            context: tokenized text using gpt/gpt2 tokenizer
            length: length of generated sequence.
            device: torch.device object.
            temperature >0: used to control the randomness of predictions by scaling the logits before applying softmax.
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
    """
    
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0)
    generated = context
    with torch.no_grad():  
        for _ in tnrange(length):
            inputs = {'input_ids': generated}
            outputs = model(**inputs)
            next_token_logits = outputs[0][0, -1, :] / temperature
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
    return generated


def generate_sample(data, tokenizer, model, num=1, eval_step=False, length=SEQ_LEN, temperature=1, top_k=10, top_p=0.5, device=torch.device(device)):

    for i in range(num):
        sample = data[randint(0, len(data) - 1)]
        idx = sample['post_len']
        context = sample['post_com'][:idx]
        summary = sample['post_com'][idx+1:][:100]
        generated_text = sample_seq(model, context, length, device, temperature, top_k, top_p)
        generated_text = generated_text[0, len(context):]
        text = tokenizer.convert_ids_to_tokens(generated_text, skip_special_tokens=True)
        text = tokenizer.convert_tokens_to_string(text)
        text = re.sub(r'\s+', ' ', text)
        text = cut(re.sub(r'\s+', ' ', text), randint(1, 2))
        if eval_step==False:
            print('\n\n')
            print('new_post:')
            print(f'<<<{tokenizer.decode(context)}>>>', end='\n\n')
            print("generated_comment:")
            print(f'<<<{text}>>>', end='\n\n')
            print('actual_comment:')
            print(f'<<<{tokenizer.decode(summary)}>>>', end='\n\n')
        else:
            print("generated_comment:")
            print(f'<<<{text}>>>', end='\n\n')
            print('\n\n')


def evaluate(model, eval_dataset, ignore_index, optimizer, global_step=None):
  torch.save({
        'epoch': 0,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, CP_PATH)


  if not os.path.exists("./output"):
    os.mkdir("./output")
  eval_output_dir = "./output"

  results = {}
  eval_sampler = SequentialSampler(eval_dataset)
  eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=1)
  loss_fct = CrossEntropyLoss() # ignore_index=ignore_idx

  eval_loss = 0.0
  nb_eval_steps = 0
  model.eval()

  for batch in tqdm(eval_dataloader, desc="Evaluating"):
    inputs, labels = torch.tensor(batch['post_com']).to(torch.device(device)), torch.tensor(batch['post_com']).to(torch.device(device))
    with torch.no_grad():
      logits = model(inputs)[0]
      idx = batch['post_len'].item()
      shift_logits = logits[..., idx:-1, :].contiguous()
      shift_labels = labels[..., idx+1:].contiguous()
      lm_loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
      eval_loss += lm_loss.mean().item()
    nb_eval_steps += 1

  eval_loss = eval_loss / nb_eval_steps
  perplexity = torch.exp(torch.tensor(eval_loss))
  result = {
      "perplexity": perplexity
      }
  print("perplexity:", perplexity.item())
  if global_step:
    for key in sorted(result.keys()):
      print("time = %s, %s = %s, step = %s\n" % (datetime.now().strftime("%d/%m/%Y %H:%M:%S"), key, str(result[key]), str(global_step)))

  return result         

In [63]:
global_step = 0
tr_loss, logging_loss = 0.0, 0.0
model.zero_grad()
train_iterator = tnrange(1, desc="Epoch")
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [ ]:

for _ in train_iterator:
  epoch_iterator = tqdm(train_dl, desc="Training")
  for step, batch in enumerate(epoch_iterator):
    
    inputs, labels = torch.tensor(batch['post_com']), torch.tensor(batch['post_com'])
    inputs = inputs.to(torch.device(device))
    labels = labels.to(torch.device(device))
    model.train()

    
    outp = model(inputs)
    logits = outp[0] 

    idx = batch['post_len'].item()
    shift_logits = logits[..., idx:-1, :].contiguous()
    shift_labels = labels[..., idx+1:].contiguous()

    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    loss = loss / 32
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    tr_loss += loss.item()

    if (step + 1) % 128 == 0:
      optimizer.step()
      scheduler.step()
      model.zero_grad()
      global_step += 1

      logging_loss = tr_loss
      print("loss:", loss.item(), end='\n\n')
      with torch.no_grad():
        if (step + 1) % 128 == 0:

            generate_sample(valid_dataset_paired, tokenizer, model, num=1, eval_step=False, device=torch.device(device))
        if (step + 1) % 2048 == 0:
            results = evaluate(model, valid_dataset_paired, ignore_idx, optimizer, global_step)
            for key, value in results.items():
                print('After', global_step+1,'updates: ', end='\n\n')
            generate_sample(valid_dataset_paired, tokenizer, model, num=1, eval_step=True, device=torch.device(device))

torch.save(model.state_dict(), END_PATH)
model.config.to_json_file('config.json')

##  Сохраняем

In [ ]:
torch.save({
        'epoch': 0,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, CP_PATH)

In [ ]:
torch.save(model, 'drive/MyDrive/Colab Notebooks/neuro_data/cp/rus04.pth')

In [ ]:
torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        }, 'drive/MyDrive/Colab Notebooks/rus_state01.pth')